In [1]:
import pandas as pd

In [2]:
df_import = pd.read_csv('../data/raw/I_TradeData_5_6_2025_3_32_56.csv', encoding='ISO-8859-1')
df_export = pd.read_csv('../data/raw/E_TradeData_5_6_2025_3_32_1.csv', encoding='ISO-8859-1')

In [3]:
print(df_import.columns)
print(df_import)
print(df_export.columns)
print(df_export)

Index(['typeCode', 'freqCode', 'refPeriodId', 'refYear', 'refMonth', 'period',
       'reporterCode', 'reporterISO', 'reporterDesc', 'flowCode', 'flowDesc',
       'partnerCode', 'partnerISO', 'partnerDesc', 'partner2Code',
       'partner2ISO', 'partner2Desc', 'classificationCode',
       'classificationSearchCode', 'isOriginalClassification', 'cmdCode',
       'cmdDesc', 'aggrLevel', 'isLeaf', 'customsCode', 'customsDesc',
       'mosCode', 'motCode', 'motDesc', 'qtyUnitCode', 'qtyUnitAbbr', 'qty',
       'isQtyEstimated', 'altQtyUnitCode', 'altQtyUnitAbbr', 'altQty',
       'isAltQtyEstimated', 'netWgt', 'isNetWgtEstimated', 'grossWgt',
       'isGrossWgtEstimated', 'cifvalue', 'fobvalue', 'primaryValue',
       'legacyEstimationFlag', 'isReported', 'isAggregate'],
      dtype='object')
   typeCode  freqCode  refPeriodId  refYear  refMonth  period reporterCode  \
C         A  20230101         2023       52      2023       8          ALB   
C         A  20230101         2023       52

In [4]:
df_import_clean = df_import[['reporterCode','reporterISO', 'fobvalue']].copy()
df_import_clean['fobvalue'] = pd.to_numeric(df_import_clean['fobvalue'], errors='coerce').fillna(0)

df_export_clean = df_export[['reporterISO','reporterDesc', 'fobvalue']].copy()
df_export_clean['fobvalue'] = pd.to_numeric(df_export_clean['fobvalue'], errors='coerce').fillna(0)

In [5]:
df_import_clean = df_import_clean.rename(columns={
    'reporterISO': 'reporter',
    'fobvalue': 'Import'
})

df_export_clean = df_export_clean.rename(columns={
    'reporterISO': 'reporterCode',
    'reporterDesc': 'reporter',
    'fobvalue': 'Export'
})

In [6]:
print(df_import_clean)
print(df_export_clean)

   reporterCode             reporter        Import
C           ALB              Albania  1.286637e+06
C           AND              Andorra  1.989116e+05
C           AGO               Angola  6.704016e+06
C           ATG  Antigua and Barbuda  1.987073e+04
C           AZE           Azerbaijan  4.120295e+05
..          ...                  ...           ...
C           BFA         Burkina Faso  1.810970e+08
C           URY              Uruguay  2.150008e+08
C           UZB           Uzbekistan  1.369807e+09
C           WSM                Samoa  5.445088e+06
C           ZMB               Zambia  1.773370e+08

[1054 rows x 3 columns]
    reporterCode             reporter        Export
0            ALB              Albania  6.374702e+04
1            AND              Andorra  3.654598e+04
2            AGO               Angola  4.685990e+02
3            ATG  Antigua and Barbuda  1.523260e+02
4            AZE           Azerbaijan  1.281059e+04
..           ...                  ...           ...

In [7]:
df_import_agg = df_import_clean.groupby('reporterCode')['Import'].sum().reset_index()
df_export_agg = df_export_clean.groupby('reporterCode')['Export'].sum().reset_index()

In [8]:
print(df_import_agg)
print(df_export_agg)

    reporterCode        Import
0            ABW  3.692475e+07
1            AGO  3.267896e+08
2            ALB  2.380411e+08
3            AND  4.504079e+07
4            ARE  6.056440e+09
..           ...           ...
149          VNM  4.046772e+09
150          WSM  6.376920e+06
151          ZAF  2.339572e+09
152          ZMB  2.487578e+08
153          ZWE  2.144565e+08

[154 rows x 2 columns]
    reporterCode        Export
0            ABW  2.067436e+05
1            AGO  8.291631e+03
2            ALB  4.489982e+06
3            AND  6.592088e+04
4            ARE  1.855188e+09
..           ...           ...
144          VNM  2.430503e+08
145          WSM  1.239293e+05
146          ZAF  4.487443e+08
147          ZMB  6.393296e+06
148          ZWE  4.536038e+06

[149 rows x 2 columns]


In [9]:
df_merged = pd.merge(df_import_agg, df_export_agg, on='reporterCode', how='outer').fillna(0)
print(df_merged)

    reporterCode        Import        Export
0            ABW  3.692475e+07  2.067436e+05
1            AGO  3.267896e+08  8.291631e+03
2            ALB  2.380411e+08  4.489982e+06
3            AND  4.504079e+07  6.592088e+04
4            ARE  6.056440e+09  1.855188e+09
..           ...           ...           ...
149          VNM  4.046772e+09  2.430503e+08
150          WSM  6.376920e+06  1.239293e+05
151          ZAF  2.339572e+09  4.487443e+08
152          ZMB  2.487578e+08  6.393296e+06
153          ZWE  2.144565e+08  4.536038e+06

[154 rows x 3 columns]


In [10]:
df_merged['Total'] = df_merged['Import'] + df_merged['Export']

df_merged = df_merged[['reporterCode', 'Total', 'Import', 'Export']]

print(df_merged)

    reporterCode         Total        Import        Export
0            ABW  3.713150e+07  3.692475e+07  2.067436e+05
1            AGO  3.267978e+08  3.267896e+08  8.291631e+03
2            ALB  2.425311e+08  2.380411e+08  4.489982e+06
3            AND  4.510671e+07  4.504079e+07  6.592088e+04
4            ARE  7.911628e+09  6.056440e+09  1.855188e+09
..           ...           ...           ...           ...
149          VNM  4.289822e+09  4.046772e+09  2.430503e+08
150          WSM  6.500849e+06  6.376920e+06  1.239293e+05
151          ZAF  2.788317e+09  2.339572e+09  4.487443e+08
152          ZMB  2.551511e+08  2.487578e+08  6.393296e+06
153          ZWE  2.189925e+08  2.144565e+08  4.536038e+06

[154 rows x 4 columns]


In [11]:
output_path = '../data/clean/trade_data.js'
with open(output_path, 'w', encoding='utf-8') as f:
    f.write('export const tradeData = {\n')
    for _, row in df_merged.iterrows():
        country = row['reporterCode'].replace('"', '\\"')
        total = round(row['Total'], 2)
        imp = round(row['Import'], 2)
        exp = round(row['Export'], 2)
        line = f'    "{country}": {{ total: {total}, import: {imp}, export: {exp} }},\n'
        f.write(line)
    f.write('};\n')

print("Done")


Done
